# [Transfer Learning in TensorFlow](https://kwotsin.github.io/tech/2017/02/11/transfer-learning.html)
# Using a Pre-trained Inception-Resnet-V2 Model  
# by Kwot Sin

one thing that I find really useful in using TensorFlow-slim over other deep learning libraries is the ready access to the best pretrained models offered by Google.


我看 Kwot Sin 這篇 blog 文說明的是這個 source code 檔:<br>
`%USERPROFILE%\Documents\GitHub\ML\transfer_learning_tutorial\train_flowers.py`
--> 跑跑看。。。

In [5]:
# These modules no problem I am sure
import os
import time
import tensorflow as tf
import os
import time
# done ! as anticipated

In [6]:
# These moduels , I'd like to impor them one after one
from tensorflow.contrib.framework.python.ops.variables import get_or_create_global_step

In [7]:
from tensorflow.python.platform import tf_logging as logging

In [8]:
import inception_preprocessing
# 直接完成，可能我先前玩過 inception，已經 install 好了？

In [9]:
from inception_resnet_v2 import inception_resnet_v2, inception_resnet_v2_arg_scope
# 直接完成，幾乎沒花時間

In [10]:
slim = tf.contrib.slim

In [ ]:
# well all done very smoothly. Now I'd like to check them out by using peforth

In [11]:
peforth.ok(loc=locals(), glo=globals(), cmd = '''
    py> tos()[0] value locals
    py> pop()[1] value globls
    ''')

OK .s
empty

OK words
code end-code \ // <selftest> </selftest> bye /// immediate stop compyle trim indent -indent <py> </py> </pyV> words . cr help interpret-only compile-only literal reveal privacy (create) : ; ( BL CR word ' , [compile] py: py> py:~ py>~ 0branch here! here swap ! @ ? >r r> r@ drop dup over 0< + * - / 1+ 2+ 1- 2- compile if then compiling char last version execute cls private nonprivate (space) exit ret rescan-word-hash (') branch bool and or not (forget) AND OR NOT XOR true false "" [] {} none >> << 0= 0> 0<> 0<= 0>= = == > < != >= <= abs max min doVar doNext depth pick roll space [ ] colon-word create (marker) marker next abort alias <> public nip rot -rot 2drop 2dup invert negate within ['] allot for begin until again ahead never repeat aft else while ?stop ?dup variable +! chars spaces .( ." .' s" s' s` does> count accept <accept> nop </accept>  refill [else] [if] [then] (::) (:>) :: :> ::~ :>~ "msg"abort abort" "msg"?abort ?abort" '<text> (<text>) <text> </text

從下面這段 code 看來，我從 [tensorflow-for-poets-2](https://github.com/hcchengithub/tensorflow-for-poets-2) 學來的 [AILAB 「剪刀、石頭、布」](https://github.com/hcchengithub/ailab_RockPaperScissors) 精簡很多。作者 Google codelabs [MarkDaoust](https://github.com/MarkDaoust) 的 `script/show_image.py` 把工作都做好了，直接帶我去應用。如今來讀 Kwot Sin 的文章，等於是帶我回頭理解前例的 source code.

下面這些 dataset 相關的 code 會出 error 

In [12]:
#================ DATASET INFORMATION ======================
#State dataset directory where the tfrecord files are located
dataset_dir = './data'  # was '.' 我覺得不好，改到 data 下

#State where your log file is at. If it doesn't exist, create it.
log_dir = './log'

#State where your checkpoint file is
checkpoint_file = './inception_resnet_v2_2016_08_30.ckpt'

#State the image size you're resizing your images to. We will use the default inception size of 299.
image_size = 299

#State the number of classes to predict: 玩 inception v3、MobileNet 都不用管這個，可能是老師處理的
num_classes = 5

#State the labels file and read it
labels_file = './labels.txt'
labels = open(labels_file, 'r')

#Create a dictionary to refer each label to their string name
labels_to_name = {}
for line in labels:
    label, string_name = line.split(':')
    string_name = string_name[:-1] #Remove newline
    labels_to_name[int(label)] = string_name

#Create the file pattern of your TFRecord files so that it could be recognized later on
file_pattern = 'flowers_%s_*.tfrecord'

#Create a dictionary that will help people understand your dataset better. This is required by the Dataset class later.
items_to_descriptions = {
    'image': 'A 3-channel RGB coloured flower image that is either tulips, sunflowers, roses, dandelion, or daisy.',
    'label': 'A label that is as such -- 0:daisy, 1:dandelion, 2:roses, 3:sunflowers, 4:tulips'
}

FileNotFoundError: [Errno 2] No such file or directory: './labels.txt'

In [ ]:
#================= TRAINING INFORMATION ==================
#State the number of epochs to train
num_epochs = 70

#State your batch size
batch_size = 10

#Learning rate information and configuration (Up to you to experiment)
initial_learning_rate = 0.0002
learning_rate_decay_factor = 0.7
num_epochs_before_decay = 2

In [ ]:
#First check whether the split_name is train or validation
if split_name not in ['train', 'validation']:
    raise ValueError('The split_name %s is not recognized.\
    Please input either train or validation as the split_name' % (split_name))

#Create the full path for a general file_pattern to locate the tfrecord_files
file_pattern_path = os.path.join(dataset_dir, file_pattern % (split_name))

In [ ]:
#Count the total number of examples in all of these shard 
num_samples = 0
file_pattern_for_counting = 'flowers_' + split_name
tfrecords_to_count = [os.path.join(dataset_dir, file) for file in os.listdir(dataset_dir) if\                                                       file.startswith(file_pattern_for_counting)]
for tfrecord_file in tfrecords_to_count:
    for record in tf.python_io.tf_record_iterator(tfrecord_file):
        num_samples += 1

In [ ]:
def get_split(split_name, dataset_dir, file_pattern=file_pattern):
    """
    Obtains the split - training or validation - to create a Dataset class for feeding the examples into a queue later
    on. This function will set up the decoder and dataset information all into one Dataset class so that you can avoid
    the brute work later on.
    
    Your file_pattern is very important in locating the files later. 

    INPUTS:
        - split_name(str): 'train' or 'validation'. Used to get the correct data split of tfrecord files
        - dataset_dir(str): the dataset directory where the tfrecord files are located
        - file_pattern(str): the file name structure of the tfrecord files in order to get the correct data

    OUTPUTS:
    - dataset (Dataset): A Dataset class object where we can read its various components for easier batch creation.
    """
    #First check whether the split_name is train or validation
    if split_name not in ['train', 'validation']:
        raise ValueError(\
        'The split_name %s is not recognized. Please input either train or validation as the split_name'\
        % (split_name))

    #Create the full path for a general file_pattern to locate the tfrecord_files
    file_pattern_path = os.path.join(dataset_dir, file_pattern % (split_name))

    #Count the total number of examples in all of these shard
    num_samples = 0
    file_pattern_for_counting = 'flowers_' + split_name
    tfrecords_to_count = [os.path.join(dataset_dir, file) for file in os.listdir(dataset_dir)\
                         if file.startswith(file_pattern_for_counting)]
    for tfrecord_file in tfrecords_to_count:
        for record in tf.python_io.tf_record_iterator(tfrecord_file):
            num_samples += 1

    #Create a reader, which must be a TFRecord reader in this case
    reader = tf.TFRecordReader

    #Create the keys_to_features dictionary for the decoder
    keys_to_features = {
      'image/encoded': tf.FixedLenFeature((), tf.string, default_value=''),
      'image/format': tf.FixedLenFeature((), tf.string, default_value='jpg'),
      'image/class/label': tf.FixedLenFeature(
          [], tf.int64, default_value=tf.zeros([], dtype=tf.int64)),
    }

    #Create the items_to_handlers dictionary for the decoder.
    items_to_handlers = {
    'image': slim.tfexample_decoder.Image(),
    'label': slim.tfexample_decoder.Tensor('image/class/label'),
    }

    #Start to create the decoder
    decoder = slim.tfexample_decoder.TFExampleDecoder(keys_to_features, items_to_handlers)

    #Create the labels_to_name file
    labels_to_name_dict = labels_to_name

    #Actually create the dataset
    dataset = slim.dataset.Dataset(
        data_sources = file_pattern_path,
        decoder = decoder,
        reader = reader,
        num_readers = 4,
        num_samples = num_samples,
        num_classes = num_classes,
        labels_to_name = labels_to_name_dict,
        items_to_descriptions = items_to_descriptions)

    return dataset